<img src="https://10botics.com/logo_jnb.png" width="300"/>

# Coral common

This files import libraries and contains common functions needed by the squat counter

In [ ]:
# Import the camera functions

# %run coral_common.ipynb

In [ ]:
from IPython.display import display, Image
import IPython
import ipywidgets as widgets
import threading
import cv2
import numpy as np

def putText(frame, text, text_color = (0, 255, 0), text_position = (50, 50)):
    # Define the text properties
    font = cv2.FONT_HERSHEY_SIMPLEX
    text_scale = 0.65
    text_thickness = 2

    # Add text annotation on the frame
    cv2.putText(frame, text, text_position, font, text_scale, text_color, text_thickness)

def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle > 180.0:
        angle = 360-angle
        
    return angle

def calculate_hip_joint_angle(keypoints_with_scores, keypoint_threshold = 0.2): 
    
    # get the position of keypoints from MoveNet output
    a1y, a1x, a1s = keypoints_with_scores[KEYPOINT_DICT["left_shoulder"]]
    a2y, a2x, a2s = keypoints_with_scores[KEYPOINT_DICT["right_shoulder"]]
    b1y, b1x, b1s = keypoints_with_scores[KEYPOINT_DICT["left_hip"]]
    b2y, b2x, b2s = keypoints_with_scores[KEYPOINT_DICT["right_hip"]]
    c1y, c1x, c1s = keypoints_with_scores[KEYPOINT_DICT["left_knee"]]
    c2y, c2x, c2s = keypoints_with_scores[KEYPOINT_DICT["right_knee"]]

    # calculate angle of left and right body respectively
    angle1 = calculate_angle( (a1y, a1x), (b1y, b1x), (c1y, c1x) )
    angle2 = calculate_angle( (a2y, a2x), (b2y, b2x), (c2y, c2x) )

    # if confident score of keypoints are all above threshold, return the midpoint of two angle
    # otherwise, return None
    if (a1s>keypoint_threshold)*(b1s>keypoint_threshold)*(c1s>keypoint_threshold):
        return (angle1 + angle2) / 2
    else:
        return None



def calculate_knee_joint_angle(keypoints_with_scores, keypoint_threshold = 0.2): 

    # get the position of keypoints from MoveNet output
    a1y, a1x, a1s = keypoints_with_scores[KEYPOINT_DICT["left_hip"]]
    a2y, a2x, a2s = keypoints_with_scores[KEYPOINT_DICT["right_hip"]]
    b1y, b1x, b1s = keypoints_with_scores[KEYPOINT_DICT["left_knee"]]
    b2y, b2x, b2s = keypoints_with_scores[KEYPOINT_DICT["right_knee"]]
    c1y, c1x, c1s = keypoints_with_scores[KEYPOINT_DICT["left_ankle"]]
    c2y, c2x, c2s = keypoints_with_scores[KEYPOINT_DICT["right_ankle"]]

    # calculate angle of left and right body respectively
    angle1 = calculate_angle( (a1y, a1x), (b1y, b1x), (c1y, c1x) )
    angle2 = calculate_angle( (a2y, a2x), (b2y, b2x), (c2y, c2x) )

    # if confident score of keypoints are all above threshold, return the midpoint of two angle
    # otherwise, return None
    if (a1s>keypoint_threshold)*(b1s>keypoint_threshold)*(c1s>keypoint_threshold):
        return (angle1 + angle2) / 2
    else:
        return None


def createStopButton():
    return  widgets.ToggleButton(
        value=False,
        description='Stop',
        disabled=False,
        button_style='danger', # 'success', 'info', 'warning', 'danger' or ''
        tooltip='Description',
        icon='square' # (FontAwesome names without the `fa-` prefix)
    )

KEYPOINT_DICT = {
    'nose': 0,
    'left_eye': 1,
    'right_eye': 2,
    'left_ear': 3,
    'right_ear': 4,
    'left_shoulder': 5,
    'right_shoulder': 6,
    'left_elbow': 7,
    'right_elbow': 8,
    'left_wrist': 9,
    'right_wrist': 10,
    'left_hip': 11,
    'right_hip': 12,
    'left_knee': 13,
    'right_knee': 14,
    'left_ankle': 15,
    'right_ankle': 16
}

# Display function
# ================
import time

def view( button, source=0, rotate=0, process=(lambda x:x)):
    """
    Args:
    button: An IPywidget button to control the function. The display stops if button.value == True.
    source: An optional integer or filename to specify the source of the video stream.
    rotate: optional integer. Set to 0 by default (not rotated). Set to 1 will rotate by 90 deg.
    process: optional function that processes each frame through process(frame) before displaying.
        Set to identity function by default.
    """
    global time_process, start_time
    time_process = []
    
    display_handle=display("Please wait...", display_id=True)
    
    cap = cv2.VideoCapture(source)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

    start_time = time.time()
    while True:
        start_time_process = time.time()
        ret, frame = cap.read()
        if button.value==True or not ret:
            cap.release()
            display_handle.update(None)
            button.value = False
            break

        #frame = cv2.flip(frame, 1)  # if the camera mirrors your image

        # Rotate the frame if rotate==1
        if rotate:
            # frame = cv2.transpose(frame)
            frame = cv2.flip(frame, 0)
        
        frame = process(frame)
        
        _, frame = cv2.imencode('.jpeg', frame)
        display_handle.update(IPython.display.Image(data=frame.tobytes()))

        time_process.append(time.time() - start_time_process)
        

In [ ]:
import PIL
from PIL import Image
from PIL import ImageDraw
from pycoral.adapters import common
from pycoral.utils.edgetpu import make_interpreter

interpreter = make_interpreter("movenet_single_pose_lightning_ptq_edgetpu.tflite")
interpreter.allocate_tensors()

_NUM_KEYPOINTS = 17

def get_result_from_image_with_model(input, interpreter, output=None):
    '''
    This function takes an input image and applies a pre-trained pose estimation model 
    to extract the position of human body keypoints in the image. 
    The TensorFlow Lite interpreter object is used to run inference on the input image
    using the pre-trained model.
    If the optional output argument is provided, the function also saves the input image
    with the detected keypoints drawn on it to the specified file path.

    Args:
    input: An input image file path or a numpy array representing an image.
    interpreter: A TensorFlow Lite interpreter object.
    output (optional): A file path to save the output image with keypoints drawn on it.

    Return:
    Array of keypoints. Each keypoint is represented as a triplet of [y, x, score].
    '''
    # load the input image using PIL
    if isinstance(input, str):
        img = PIL.Image.open(input)
    elif isinstance(input, np.ndarray):
        img = PIL.Image.fromarray(cv2.cvtColor(input, cv2.COLOR_BGR2RGB))

    # resize the image to model required size
    resized_img = img.resize(common.input_size(interpreter), PIL.Image.Resampling.LANCZOS)

    # load the resized image to interpreter
    common.set_input(interpreter, resized_img)

    # conduct the inference
    interpreter.invoke()

    # reshape and assign the inference result to variable `pose`
    keypoints_with_scores = common.output_tensor(interpreter, 0).copy().reshape(_NUM_KEYPOINTS, 3)

    # draw the keypoints and save the image (if specified `output`)
    if output:
        draw = ImageDraw.Draw(img)
        width, height = img.size
        for i in range(0, _NUM_KEYPOINTS):
            draw.ellipse(
            xy=[
                keypoints_with_scores[i][1] * width - 2, keypoints_with_scores[i][0] * height - 2,
                keypoints_with_scores[i][1] * width + 2, keypoints_with_scores[i][0] * height + 2
            ],
            fill=(255, 0, 0))
        img.save(output)
        
    return keypoints_with_scores

<hr/>

## Congratulation! You have finished this chapter.

This jupyter notebook is created by 10Botics. <br>
For permission to use in school, please contact info@10botics.com <br>
All rights reserved. 2024.